<a href="https://colab.research.google.com/github/HJoon-K/Find_Dust/blob/kim/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

## 서울시 구별 미세먼지 현황 (2016 - 2020)

In [ ]:
files = ['dust2020.csv','dust2019.csv','dust2018.csv','dust2017.csv','dust2016.csv']
dust = pd.DataFrame()
for idx, file in enumerate(files):
    dust1 = pd.read_csv(f'/content/drive/MyDrive/Colab Notebooks/dustdata/{file}')
    if idx == 0:
        dust = dust.append(dust1)
    else:
        dust = dust.append(dust1, ignore_index=True)

dust.tail()

,date,gu,PM10,PM25,자치구,연도,미세먼지나쁨일수
36493,20161231.0,마포구,56.0,47.0,NaN,NaN,NaN
36494,20161231.0,양천구,61.0,37.0,NaN,NaN,NaN
36495,20161231.0,강동구,63.0,41.0,NaN,NaN,NaN
36496,20161231.0,영등포구,56.0,36.0,NaN,NaN,NaN
36497,20161231.0,은평구,51.0,33.0,NaN,NaN,NaN


In [ ]:
dust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36498 entries, 0 to 36497
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      36473 non-null  float64
 1   gu        36473 non-null  object 
 2   PM10      36473 non-null  float64
 3   PM25      36473 non-null  float64
 4   자치구       25 non-null     object 
 5   연도        25 non-null     float64
 6   미세먼지나쁨일수  25 non-null     float64
dtypes: float64(5), object(2)
memory usage: 1.9+ MB


In [ ]:
dust.to_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/dust.csv',index=False)

In [ ]:
dust2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/dust.csv')

In [ ]:
dust2.info()
dust2['date']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36498 entries, 0 to 36497
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      36473 non-null  float64
 1   gu        36473 non-null  object 
 2   PM10      36473 non-null  float64
 3   PM25      36473 non-null  float64
 4   자치구       25 non-null     object 
 5   연도        25 non-null     float64
 6   미세먼지나쁨일수  25 non-null     float64
dtypes: float64(5), object(2)
memory usage: 1.9+ MB


0        20201231.0
1        20201231.0
2        20201231.0
3        20201231.0
4        20201231.0
            ...    
36493    20161231.0
36494    20161231.0
36495    20161231.0
36496    20161231.0
36497    20161231.0
Name: date, Length: 36498, dtype: float64

In [ ]:
# 구별 연도별 (2016 - 2020)
dust_yr = dust2[(dust2['PM10'] > 80) | (dust2['PM25'] > 35)]
dust_yr['year'] = pd.to_datetime(dust_yr['date'].astype(str), format='%Y%m%d').dt.year
dust_year = dust_yr.groupby(['gu','year'])['date'].count()
dust_year = pd.DataFrame(dust_year).reset_index()
dust_year.rename(columns={'gu':'자치구', 'year':'연도' , 'date':'미세먼지나쁨일수'}, inplace=True)
dust_year.to_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/dust_year.csv', index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
dust_year

,자치구,연도,미세먼지나쁨일수
0,강남구,2016,90
1,강남구,2017,84
2,강남구,2018,64
3,강남구,2020,45
4,강동구,2016,87
...,...,...,...
95,중구,2020,63
96,중랑구,2016,87
97,중랑구,2017,80
98,중랑구,2018,54


In [ ]:
# 구별 미세먼지 or 초미세먼지 : 나쁨 이상 횟수
dust5 = dust2[(dust2['PM10'] > 80) | (dust2['PM25'] > 35)]
dust_gu = pd.DataFrame(dust5['gu'].value_counts())
dust_gu = dust_gu.reset_index()
dust_gu.rename(columns=({'index':'gu', 'gu':'count'}), inplace=True)
dust_gu.to_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/dust_gu_pm10_pm25.csv', index=False)

In [ ]:
dust_gu

,gu,count
0,마포구,321
1,영등포구,300
2,양천구,297
3,관악구,296
4,구로구,290
5,광진구,289
6,강동구,286
7,강남구,283
8,종로구,276
9,서초구,273


In [ ]:
# 구별/월별 미세먼지 or 초미세먼지 : 나쁨 이상 횟수
dust5['month'] = pd.to_datetime(dust5['date'].astype(str), format='%Y%m%d').dt.month
dust5_gp = dust5.groupby(['gu','month'])['date'].count()
dust5_gp = dust5_gp.reset_index()
dust5_gp.rename(columns={'date':'count'}, inplace=True)
dust5_gp.to_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/dust_gu_m_pm10_pm25.csv', index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## 서울시 구별 미세먼지 현황 (2019)

In [ ]:
dust19 = dust_year[dust_year['연도'] == 2019]
dust19.to_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/dust2019.csv', index=False)

In [ ]:
dust1619 = dust_year[~(dust_year['연도'] == 2020)]
dust1619.to_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/dust1619.csv', index=False)

## 서울시 구별 미세먼지 현황 (2020)

In [ ]:
dust20 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/dust2020.csv')
# 구별 미세먼지 or 초미세먼지 : 나쁨 이상 횟수
dust_t = dust20[(dust20['PM10'] > 80) | (dust20['PM25'] > 35)]
dust20_gu = pd.DataFrame(dust_t['gu'].value_counts())
dust20_gu = dust20_gu.reset_index()
dust20_gu.rename(columns=({'index':'gu', 'gu':'count'}), inplace=True)
dust20_gu.to_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/dust20_gu_pm10_pm25.csv', index=False)
dust20_gu

,gu,count
0,서초구,64
1,중구,63
2,구로구,63
3,강동구,58
4,동작구,57
5,양천구,56
6,노원구,55
7,마포구,53
8,강북구,52
9,금천구,51


### 2019 

In [ ]:
dust19 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/dust2019.csv')
# 구별 미세먼지 or 초미세먼지 : 나쁨 이상 횟수
dust_t = dust19[(dust19['PM10'] > 80) | (dust19['PM25'] > 35)]
dust19_gu = pd.DataFrame(dust_t['gu'].value_counts())
dust19_gu = dust19_gu.reset_index()
dust19_gu.rename(columns=({'index':'자치구', 'gu':'미세먼지나쁨일수'}), inplace=True)
dust19_gu.to_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/dust19_gu_pm10_pm25.csv', index=False)
dust19_gu

,자치구,미세먼지나쁨일수
0,영등포구,86
1,성동구,81
2,마포구,80
3,양천구,76
4,관악구,76
5,서초구,74
6,광진구,74
7,구로구,71
8,동작구,71
9,강동구,70


## 서울시 구별 공원면적 / 공원율
* 결과물 : seoul_park.csv

In [ ]:
park = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/park2020.csv')
park.head()

,자치구별(1),자치구별(2),구분별(1),2020
0,합계,소계,행정구역면적,605228541
1,합계,소계,공원면적,172317390
2,합계,소계,자연공원(국립)면적,37345000
3,합계,소계,도시자연공원구역면적,69222532
4,합계,종로구,행정구역면적,23913168


In [ ]:
park.drop('자치구별(1)',axis=1, inplace=True)

In [ ]:
park = park[4:].reset_index()

In [ ]:
park.drop(columns='index', axis=1, inplace=True)

In [ ]:
pt_park = park.pivot_table('2020', '자치구별(2)','구분별(1)').reset_index()
pt_park = pt_park.rename(columns={'자치구별(2)':'gu'})

In [ ]:
pt_park['자연공원(국립)면적'].fillna(0, inplace=True)
pt_park = pt_park[~(pt_park.gu == '서울대공원')].reset_index().drop('index', axis=1)

In [ ]:
pt_park

구분별(1),gu,공원면적,도시자연공원구역면적,자연공원(국립)면적,행정구역면적
0,강남구,7768748.0,4065697.0,0.0,39501006.0
1,강동구,3362412.0,758183.0,0.0,24590480.0
2,강북구,14424502.0,412374.0,11899000.0,23600355.0
3,강서구,4512052.0,975028.0,0.0,41436848.0
4,관악구,12431424.0,10418474.0,0.0,29569356.0
5,광진구,3451040.0,2097470.0,0.0,17062939.0
6,구로구,3641446.0,2003079.0,0.0,20119910.0
7,금천구,2774378.0,2210642.0,0.0,13020252.0
8,노원구,14741807.0,12798824.0,0.0,35437648.0
9,도봉구,10160375.0,335258.0,8703000.0,20651082.0


In [ ]:
park1 = pt_park
park1

구분별(1),gu,공원면적,도시자연공원구역면적,자연공원(국립)면적,행정구역면적
0,강남구,7768748.0,4065697.0,0.0,39501006.0
1,강동구,3362412.0,758183.0,0.0,24590480.0
2,강북구,14424502.0,412374.0,11899000.0,23600355.0
3,강서구,4512052.0,975028.0,0.0,41436848.0
4,관악구,12431424.0,10418474.0,0.0,29569356.0
5,광진구,3451040.0,2097470.0,0.0,17062939.0
6,구로구,3641446.0,2003079.0,0.0,20119910.0
7,금천구,2774378.0,2210642.0,0.0,13020252.0
8,노원구,14741807.0,12798824.0,0.0,35437648.0
9,도봉구,10160375.0,335258.0,8703000.0,20651082.0


In [ ]:
park1['자연공원면적'] = park1['도시자연공원구역면적'] + park1['자연공원(국립)면적']
park1['인공공원면적'] = park1['공원면적'] - park1['자연공원면적']
park1['공원율'] = park1['공원면적'] / park1['행정구역면적']

In [ ]:
park1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   gu          25 non-null     object 
 1   공원면적        25 non-null     float64
 2   도시자연공원구역면적  25 non-null     float64
 3   자연공원(국립)면적  25 non-null     float64
 4   행정구역면적      25 non-null     float64
 5   자연공원면적      25 non-null     float64
 6   인공공원면적      25 non-null     float64
 7   공원율         25 non-null     float64
dtypes: float64(7), object(1)
memory usage: 1.7+ KB


In [ ]:
park_p = park1.iloc[:,[0,4,1,5,6,7]]
park_p.to_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/seoul_park20.csv', index=False)

### 서울시 구별 공원면적 / 공원율 (2016 - 2020)

In [ ]:
park_p = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/seoul_park20.csv')
park_p = park_p.loc[:,['gu','행정구역면적']]

park_p

,gu,행정구역면적
0,강남구,39501006.0
1,강동구,24590480.0
2,강북구,23600355.0
3,강서구,41436848.0
4,관악구,29569356.0
5,광진구,17062939.0
6,구로구,20119910.0
7,금천구,13020252.0
8,노원구,35437648.0
9,도봉구,20651082.0


In [ ]:
# 컬럼 추가하고 데이터프레임 추가후 정렬 - 서울시 구별 행정구역면적(2016 - 2020)
lists = ['2016','2017','2018','2019','2020']
seoul_c = pd.DataFrame(columns=(['gu', '행정구역면적']))
for yr in lists:
    park_p['연도'] =  yr
    seoul_c = seoul_c.append(park_p, ignore_index=True)
   
seoul_c = seoul_c.sort_values(['gu','연도'])
seoul_area = seoul_c.loc[:,['gu','연도','행정구역면적']].rename(columns={'gu':'자치구'})
seoul_area.to_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/seoul_area.csv')

In [ ]:
# 데이터 set_index, reset_index, sort_values
park_t = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/park16_20.csv')
park_t.drop(['자치구별(1)','공원종류별(1)'], axis=1, inplace=True)
park_t.set_index('자치구별(2)', inplace=True)
park_t = park_t.stack().reset_index()
park_t.rename(columns={'자치구별(2)': '자치구', 'level_1': '연도', 0:'공원면적'}, inplace=True)
park_t = park_t[~(park_t['자치구'] == '서울대공원')].sort_values(['자치구','연도'])

# 행정구역면적df + 공원면적df 
park_t = pd.merge(park_t, seoul_area, on=['자치구','연도'], how='inner')
park_t['행정구역면적'] = park_t['행정구역면적'] // 1000
park_t['공원율'] = park_t['공원면적'] / park_t['행정구역면적']

# file 생성
park_t.to_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/seoul_park1620.csv', index=False)

In [ ]:
park_t

,자치구,연도,공원면적,행정구역면적,공원율
0,강남구,2016,6598.0,39501.0,0.167034
1,강남구,2017,6591.7,39501.0,0.166874
2,강남구,2018,6591.7,39501.0,0.166874
3,강남구,2019,6594.3,39501.0,0.166940
4,강남구,2020,7768.7,39501.0,0.196671
...,...,...,...,...,...
120,중랑구,2016,5211.9,18496.0,0.281785
121,중랑구,2017,5206.5,18496.0,0.281493
122,중랑구,2018,5206.3,18496.0,0.281482
123,중랑구,2019,5205.3,18496.0,0.281428


### 서울시 구별 공원면적 / 공원율 (2019)

In [ ]:
park19 = park_t[park_t['연도'] == '2019']

In [ ]:
park19.to_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/seoul_park2019.csv', index=False)

In [ ]:
park1619 = park_t[~(park_t['연도'] == '2020')]
park1619.to_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/seoul_park1619.csv', index=False)

## 서울시 구별 가로수 현황

### 2020년

In [ ]:
trees = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/trees.csv')
trees1 = trees.iloc[:,[1,2]]
trees1 = trees1.rename(columns={'자치구별(2)':'gu', '소계':'가로수량'})
trees1.to_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/seoul_trees.csv', index=False)

In [ ]:
trees1

,gu,가로수량
0,종로구,7435
1,중구,8300
2,용산구,7821
3,성동구,7927
4,광진구,7238
5,동대문구,10676
6,중랑구,8270
7,성북구,8485
8,강북구,7346
9,도봉구,9449


### 2016 - 2020년

In [ ]:
tree_t = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/tree16_20.csv')
tree_t = tree_t.drop('자치구별(1)',axis=1).set_index('자치구별(2)')
tree_t = tree_t[:25].stack().reset_index()
tree_t.rename(columns={'자치구별(2)':'자치구', 'level_1': '연도', 0:'가로수량'}, inplace=True)
tree_t = tree_t.sort_values(['자치구', '연도'])

tree_t.to_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/seoul_tree1620.csv', index=False)

### 2019


In [ ]:
tree19 = tree_t[tree_t['연도'] == '2019']
tree19

,자치구,연도,가로수량
113,강남구,2019,23349
123,강동구,2019,17152
43,강북구,2019,7789
78,강서구,2019,13719
103,관악구,2019,10154
23,광진구,2019,7223
83,구로구,2019,11548
88,금천구,2019,6603
53,노원구,2019,14489
48,도봉구,2019,9572


In [ ]:
tree19.to_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/seoul_tree2019.csv', index=False)

### 2016 - 2019

In [ ]:
tree1619 = tree_t[~(tree_t['연도'] == '2020')]
tree1619.to_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/seoul_tree1619.csv', index=False)

## 서울시 구별 도로율 (2020)


In [ ]:
road = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/road_area.csv')
road = road.rename(columns={'자치구별': 'gu', '도로면적 (㎢)': 'area', '도로율 (%)': 'rate'})
road

,gu,area,rate
0,종로구,2.701070,21.44
1,중구,1.876189,18.82
2,용산구,3.154983,22.19
3,성동구,3.024659,23.09
4,광진구,3.436834,28.83
5,동대문구,3.151074,22.16
6,중랑구,2.861614,26.01
7,성북구,4.225186,23.59
8,강북구,1.984564,20.93
9,도봉구,2.354393,19.02


In [ ]:
road.to_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/seoul_road.csv', index=False)

## 서울시 구별 도로율 (2016 - 2020)

In [ ]:
road_t = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/road16_20.csv')
road_t.drop('자치구별(1)', axis=1, inplace=True)
road_t = road_t[1:].set_index('자치구별(2)')
road_t = road_t.stack().reset_index().rename(columns={'자치구별(2)':'자치구','level_1':'연도',0:'도로율'}).sort_values(['자치구','연도'])
road_t.to_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/seoul_road1620.csv',index=False)

### 2019

In [ ]:
road19 = road_t[road_t['연도'] == '2019' ]
road19.head()

,자치구,연도,도로율
113,강남구,2019,23.58
123,강동구,2019,24.28
43,강북구,2019,20.91
78,강서구,2019,26.83
103,관악구,2019,25.41


In [ ]:
road19.to_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/seoul_road2019.csv',index=False)

### 2016 - 2019

In [ ]:
road1619 = road_t[~(road_t['연도'] == '2020')]
road1619.to_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/seoul_road1619.csv', index=False)

In [ ]:
road1619.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 110 to 33
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   자치구     100 non-null    object 
 1   연도      100 non-null    object 
 2   도로율     100 non-null    float64
dtypes: float64(1), object(2)
memory usage: 3.1+ KB


## 서울시 인구밀도

In [ ]:
popul = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/population.csv')
popul = popul.rename(columns={'구별':'gu', '인구밀도 (명/㎢)':'인구밀도'})
popul = popul.loc[:,['gu','인구밀도']]
popul

,gu,인구밀도
0,종로구,6431
1,중구,13231
2,용산구,10852
3,성동구,17359
4,광진구,20666
5,동대문구,24762
6,중랑구,21188
7,성북구,17909
8,강북구,12820
9,도봉구,15465


In [ ]:
popul.to_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/seoul_popul.csv', index=False)

## 서울시 구별 가로수 / 공원율 / 도로율 Merge 데이터처리 (2016 - 2020)

In [ ]:
dust = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/dust_year.csv')
park = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/seoul_park1620.csv')
road = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/seoul_road1620.csv')
tree = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/seoul_tree1620.csv')

In [ ]:
road = road.rename(columns={'rate':'도로율'})

In [ ]:
print(dust.info(), park.info(), road.info(), tree.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   자치구       100 non-null    object
 1   연도        100 non-null    int64 
 2   미세먼지나쁨일수  100 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 2.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125 entries, 0 to 124
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   자치구     125 non-null    object 
 1   연도      125 non-null    int64  
 2   공원면적    125 non-null    float64
 3   행정구역면적  125 non-null    float64
 4   공원율     125 non-null    float64
dtypes: float64(3), int64(1), object(1)
memory usage: 5.0+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125 entries, 0 to 124
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   자치구     125 non-null    object 
 1   연도      125 

In [ ]:
df = pd.merge(dust, park, how='inner', on=['자치구','연도']).loc[:,['자치구','연도','미세먼지나쁨일수','공원율']]
df = pd.merge(df, road, how='inner', on=['자치구','연도']).loc[:,['자치구','연도','미세먼지나쁨일수','공원율','도로율']]
df['도로율'] = df['도로율'] / 100
df = pd.merge(df, tree, how='inner', on=['자치구','연도'])
df.to_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/seoul_dust1620.csv', index=False)

In [ ]:
df

,자치구,연도,미세먼지나쁨일수,공원율,도로율,가로수량
0,강남구,2016,90,0.167034,0.2346,23654
1,강남구,2017,84,0.166874,0.2346,23729
2,강남구,2018,64,0.166874,0.2347,23451
3,강남구,2020,45,0.196671,0.2358,23479
4,강동구,2016,87,0.137641,0.2350,16414
...,...,...,...,...,...,...
95,중구,2020,63,0.317651,0.1882,8300
96,중랑구,2016,87,0.281785,0.2585,8114
97,중랑구,2017,80,0.281493,0.2597,8155
98,중랑구,2018,54,0.281482,0.2598,8166


## 서울시 구별 가로수 / 공원율 / 도로율 Merge 데이터처리 (2020)

In [ ]:
dust = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/dust20_gu_pm10_pm25.csv')
park = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/seoul_park.csv')
road = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/seoul_road.csv')
tree = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/seoul_trees.csv')
popul = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/seoul_popul.csv')
road = road.rename(columns={'rate':'도로율'})

In [ ]:
df = pd.merge(dust, park, how='inner', on='gu').loc[:,['gu','count','공원율']]
df = pd.merge(df, road, how='inner', on='gu').loc[:,['gu','count','공원율','도로율']]
df['도로율'] = df['도로율'] / 100
df = pd.merge(df, tree, how='inner', on='gu')
df = pd.merge(df, popul, how='inner', on='gu')
df.to_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/seoul_dust20.csv', index=False)

In [ ]:
df

,gu,count,공원율,도로율,가로수량,인구밀도
0,서초구,64,0.320819,0.2635,16099,8858
1,중구,63,0.317644,0.1882,8300,13231
2,구로구,63,0.180987,0.2206,11692,20933
3,강동구,58,0.136736,0.2400,17028,189
4,동작구,57,0.264690,0.1729,7226,24113
5,양천구,56,0.175208,0.2788,13509,25882
6,노원구,55,0.415993,0.2371,13276,14530
7,마포구,53,0.191417,0.2861,14558,15878
8,강북구,52,0.611199,0.2093,7346,12820
9,금천구,51,0.213082,0.2138,6495,18808


## 서울시 구별 가로수 / 공원율 / 도로율 Merge 데이터처리 (2019)

In [ ]:
dust = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/dust19_gu_pm10_pm25.csv')
park = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/seoul_park2019.csv')
road = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/seoul_road2019.csv')
tree = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/seoul_tree2019m.csv')

road = road.rename(columns={'rate':'도로율'})

In [ ]:
dust

,자치구,미세먼지나쁨일수
0,영등포구,86
1,성동구,81
2,마포구,80
3,양천구,76
4,관악구,76
5,서초구,74
6,광진구,74
7,구로구,71
8,동작구,71
9,강동구,70


In [ ]:
df = pd.merge(dust, park, how='inner', on='자치구').loc[:,['자치구','미세먼지나쁨일수','공원율']]
df = pd.merge(df, road, how='inner', on='자치구').loc[:,['자치구','미세먼지나쁨일수','공원율','도로율']]
df['도로율'] = df['도로율'] / 100
df = pd.merge(df, tree, how='inner', on='자치구')
df.to_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/seoul_dust2019.csv', index=False)

In [ ]:
df

,자치구,미세먼지나쁨일수,공원율,도로율,가로수량도로면적당
0,영등포구,86,0.124263,0.2350,4103
1,성동구,81,0.183856,0.2309,2688
2,마포구,80,0.186164,0.2837,3563
3,양천구,76,0.167308,0.2718,3729
4,관악구,76,0.371230,0.2541,3351
5,서초구,74,0.342287,0.2635,2858
6,광진구,74,0.197134,0.2883,2099
7,구로구,71,0.165818,0.2163,3654
8,동작구,71,0.280328,0.1716,2594
9,강동구,70,0.137039,0.2428,4845


### 2016 - 2019 Merge

In [ ]:
dust = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/dust1619.csv')
park = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/seoul_park1619.csv')
road = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/seoul_road1619.csv')
tree = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/seoul_tree1619.csv')

road = road.rename(columns={'rate':'도로율'})

In [ ]:
tree

,자치구,연도,가로수량
0,강남구,2016,23654
1,강남구,2017,23729
2,강남구,2018,23451
3,강남구,2019,23349
4,강동구,2016,16414
...,...,...,...
95,중구,2019,8250
96,중랑구,2016,8114
97,중랑구,2017,8155
98,중랑구,2018,8166


In [ ]:
df = pd.merge(dust, park, how='inner', on=['자치구','연도']).loc[:,['자치구','연도','미세먼지나쁨일수','공원율']]
df = pd.merge(df, road, how='inner', on=['자치구','연도']).loc[:,['자치구','연도','미세먼지나쁨일수','공원율','도로율']]
df['도로율'] = df['도로율'] / 100
df = pd.merge(df, tree, how='inner', on=['자치구','연도'])
df.to_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/seoul_dust1619.csv', index=False)

In [ ]:
df

,자치구,연도,미세먼지나쁨일수,공원율,도로율,가로수량
0,강남구,2016,90,0.167034,0.2346,23654
1,강남구,2017,84,0.166874,0.2346,23729
2,강남구,2018,64,0.166874,0.2347,23451
3,강동구,2016,87,0.137641,0.2350,16414
4,강동구,2017,69,0.137641,0.2418,16518
...,...,...,...,...,...,...
70,중구,2017,70,0.314458,0.1882,8174
71,중구,2018,50,0.314458,0.1882,8178
72,중랑구,2016,87,0.281785,0.2585,8114
73,중랑구,2017,80,0.281493,0.2597,8155


In [ ]:
road

,자치구,연도,도로율
0,강남구,2016,23.46
1,강남구,2017,23.46
2,강남구,2018,23.47
3,강남구,2019,23.58
4,강동구,2016,23.50
...,...,...,...
95,중구,2019,18.82
96,중랑구,2016,25.85
97,중랑구,2017,25.97
98,중랑구,2018,25.98


In [ ]:
tree19 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/seoul_tree2019m.csv')
road19 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/road2019.csv')
road19.drop(['자치구별(1)','2019','2019.2'],axis=1, inplace=True)
road19 = road19.rename(columns={'자치구별(2)':'자치구', '2019.1':'도로면적'})
road19 = road19[2:]

In [ ]:
tree19.head()

,자치구,연도,가로수량
0,강남구,2019,23349
1,강동구,2019,17152
2,강북구,2019,7789
3,강서구,2019,13719
4,관악구,2019,10154


In [ ]:
tree19_m = pd.merge(tree19, road19, on='자치구', how='inner').loc[:,['자치구','가로수량','도로면적']]
tree19_m['가로수량도로면적당'] = (tree19_m['가로수량'] / tree19_m['도로면적'].astype(float)).astype(int)
tree19_m = tree19_m.iloc[:,[0,3]]
tree19_m

,자치구,가로수량도로면적당
0,강남구,4096
1,강동구,4845
2,강북구,3933
3,강서구,2727
4,관악구,3351
5,광진구,2099
6,구로구,3654
7,금천구,3001
8,노원구,3783
9,도봉구,4073


In [ ]:
tree19_m.to_csv('/content/drive/MyDrive/Colab Notebooks/dustdata/seoul_tree2019m.csv', index=False)